In [76]:
import pandas as pd

In [77]:
course_metadata = pd.read_csv(r"C:\My Progress\UE Internship\Version 02\Code\csv_dataframe - Sheet1.csv") 

In [78]:
# Create a dictionary where the keys are column names 'lh0' to 'lh14'
columns = [f'lh{i}' for i in range(10)]

# Create a DataFrame with 5 rows and 15 columns
data = [[f'Row{row+1}_Col{col}' for col in range(10)] for row in range(5)]

# Create the DataFrame
timetable = pd.DataFrame(data, columns=columns)

timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9
0,Row1_Col0,Row1_Col1,Row1_Col2,Row1_Col3,Row1_Col4,Row1_Col5,Row1_Col6,Row1_Col7,Row1_Col8,Row1_Col9
1,Row2_Col0,Row2_Col1,Row2_Col2,Row2_Col3,Row2_Col4,Row2_Col5,Row2_Col6,Row2_Col7,Row2_Col8,Row2_Col9
2,Row3_Col0,Row3_Col1,Row3_Col2,Row3_Col3,Row3_Col4,Row3_Col5,Row3_Col6,Row3_Col7,Row3_Col8,Row3_Col9
3,Row4_Col0,Row4_Col1,Row4_Col2,Row4_Col3,Row4_Col4,Row4_Col5,Row4_Col6,Row4_Col7,Row4_Col8,Row4_Col9
4,Row5_Col0,Row5_Col1,Row5_Col2,Row5_Col3,Row5_Col4,Row5_Col5,Row5_Col6,Row5_Col7,Row5_Col8,Row5_Col9


In [79]:
import pandas as pd
import random
import numpy as np
from collections import Counter

# Initialize timetable with 5 days and 10 lecture halls
columns = [f'lh{i}' for i in range(10)]
data = [[None for _ in range(10)] for _ in range(5)]
timetable = pd.DataFrame(data, columns=columns)

# GA parameters
POPULATION_SIZE = 100
GENES = list(range(len(course_metadata)))  # Courses are represented by indices in course_metadata
MAX_GENERATIONS = 200
ANNEALING_THRESHOLD = 50  # Apply simulated annealing if no improvement in 50 generations

# Individual class
class Individual:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.fitness = self.calculate_fitness()

    @classmethod
    def create_gnome(cls):
        return [random.choice(GENES) for _ in range(7)]

    @classmethod
    def mutate_gene(cls):
        return random.choice(GENES)

    def mate(self, partner):
        child_chromosome = []
        for gene1, gene2 in zip(self.chromosome, partner.chromosome):
            prob = random.random()
            if prob < 0.45:
                child_chromosome.append(gene1)
            elif prob < 0.90:
                child_chromosome.append(gene2)
            else:
                child_chromosome.append(self.mutate_gene())
        return Individual(child_chromosome)

    def calculate_fitness(self):
        fitness = 0
        counts = Counter(self.chromosome)
        
        # Constraints for credit hours and avoiding overlaps
        for i, course_index in enumerate(self.chromosome):
            if course_metadata.iloc[course_index]['Credit Hours'] <= 0:
                fitness += 1
            if counts[course_index] > course_metadata.iloc[course_index]['Credit Hours']:
                fitness += counts[course_index] - course_metadata.iloc[course_index]['Credit Hours']

        return fitness

def tournament_selection(population):
    tournament = random.sample(population, k=5)
    return min(tournament, key=lambda ind: ind.fitness)

def reduce_credit_hours(course_list):
    for course in course_list:
        course_metadata.loc[course, 'Credit Hours'] -= 1

def simulated_annealing(individual, temperature=1.0, cooling_rate=0.95):
    current = individual
    best = individual
    while temperature > 0.01:
        new_individual = Individual([gene if random.random() > 0.2 else random.choice(GENES) for gene in current.chromosome])
        if new_individual.fitness < best.fitness:
            best = new_individual
        elif random.random() < np.exp((current.fitness - new_individual.fitness) / temperature):
            current = new_individual
        temperature *= cooling_rate
    return best

def initialize_population():
    # Greedy initialization for a better start
    population = []
    for _ in range(POPULATION_SIZE // 2):
        chromosome = [random.choice(GENES) for _ in range(7)]
        population.append(Individual(chromosome))
    # Adding some completely random individuals
    for _ in range(POPULATION_SIZE // 2):
        chromosome = Individual.create_gnome()
        population.append(Individual(chromosome))
    return population

def main():
    ctr = 0
    global timetable
    for day in timetable.index:
        print("Processing day:", day)
        for lh in timetable.columns:
            generation = 1
            found = False
            population = initialize_population()
            no_improvement_count = 0
            best_fitness = float('inf')
            
            while not found and generation < MAX_GENERATIONS:
                population.sort(key=lambda x: x.fitness)

                # Check for best individual
                if population[0].fitness == 0:
                    found = True
                    timetable.loc[day, lh] = population[0].chromosome
                    reduce_credit_hours(population[0].chromosome)
                    ctr+=1
                    print(ctr)
                    break

                # If no improvement in fitness, apply simulated annealing
                if population[0].fitness < best_fitness:
                    best_fitness = population[0].fitness
                    no_improvement_count = 0
                else:
                    no_improvement_count += 1
                    if no_improvement_count >= ANNEALING_THRESHOLD:
                        population[0] = simulated_annealing(population[0])
                        no_improvement_count = 0

                # New generation using elitism and mating
                new_generation = []
                new_generation.extend(population[:int(0.15 * POPULATION_SIZE)])  # Elitism
                for _ in range(int(0.7 * POPULATION_SIZE)):
                    parent1 = tournament_selection(population)
                    parent2 = tournament_selection(population)
                    child = parent1.mate(parent2)
                    new_generation.append(child)
                for _ in range(int(0.15 * POPULATION_SIZE)):
                    new_generation.append(Individual(Individual.create_gnome()))  # Random mutation
                
                population = new_generation
                generation += 1

if __name__ == "__main__":
    main()
    print(timetable)


Processing day: 0
1
2
3
4
5
6
7
8
9
10
Processing day: 1
11
12
13
14
15
16
17
18
19
20
Processing day: 2
21
22
23
24
25
26
27
28
29
30
Processing day: 3
31
32
33
34
35
36
37
38
39
40
Processing day: 4
41
42
43
                              lh0                            lh1  \
0     [41, 13, 59, 18, 56, 40, 8]     [65, 54, 98, 6, 43, 66, 3]   
1      [0, 13, 19, 43, 50, 7, 94]  [77, 100, 42, 30, 75, 86, 44]   
2   [87, 23, 101, 75, 91, 78, 25]   [98, 76, 63, 42, 98, 34, 29]   
3  [61, 101, 100, 48, 97, 66, 66]   [62, 99, 15, 56, 23, 79, 76]   
4   [67, 31, 41, 100, 41, 10, 31]   [58, 31, 53, 15, 47, 15, 24]   

                            lh2                           lh3  \
0   [83, 1, 76, 25, 17, 51, 85]   [77, 45, 4, 73, 72, 16, 57]   
1   [68, 72, 11, 57, 80, 10, 1]    [3, 70, 91, 2, 78, 30, 20]   
2  [90, 16, 18, 74, 21, 40, 39]    [48, 5, 9, 40, 84, 70, 21]   
3    [94, 6, 7, 37, 69, 49, 32]  [36, 97, 27, 84, 67, 32, 96]   
4  [36, 61, 58, 58, 47, 60, 24]                         

In [80]:
timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9
0,"[41, 13, 59, 18, 56, 40, 8]","[65, 54, 98, 6, 43, 66, 3]","[83, 1, 76, 25, 17, 51, 85]","[77, 45, 4, 73, 72, 16, 57]","[72, 5, 77, 22, 52, 11, 0]","[45, 26, 74, 2, 70, 35, 92]","[83, 57, 65, 80, 33, 38, 10]","[62, 92, 33, 23, 69, 46, 37]","[8, 87, 34, 68, 55, 22, 93]","[64, 85, 20, 71, 88, 65, 54]"
1,"[0, 13, 19, 43, 50, 7, 94]","[77, 100, 42, 30, 75, 86, 44]","[68, 72, 11, 57, 80, 10, 1]","[3, 70, 91, 2, 78, 30, 20]","[14, 2, 43, 0, 14, 53, 68]","[95, 16, 34, 50, 4, 29, 35]","[97, 83, 44, 49, 74, 95, 12]","[45, 82, 38, 95, 96, 5, 48]","[69, 52, 44, 50, 99, 9, 73]","[99, 9, 84, 38, 92, 59, 96]"
2,"[87, 23, 101, 75, 91, 78, 25]","[98, 76, 63, 42, 98, 34, 29]","[90, 16, 18, 74, 21, 40, 39]","[48, 5, 9, 40, 84, 70, 21]","[22, 54, 27, 81, 51, 12, 90]","[4, 81, 8, 29, 62, 14, 63]","[80, 52, 46, 37, 67, 28, 56]","[64, 85, 79, 33, 73, 82, 49]","[35, 27, 81, 19, 18, 32, 87]","[26, 47, 42, 30, 89, 63, 89]"
3,"[61, 101, 100, 48, 97, 66, 66]","[62, 99, 15, 56, 23, 79, 76]","[94, 6, 7, 37, 69, 49, 32]","[36, 97, 27, 84, 67, 32, 96]","[51, 75, 88, 1, 82, 19, 55]","[12, 64, 11, 36, 86, 71, 59]","[25, 90, 7, 20, 93, 89, 60]","[86, 26, 28, 17, 91, 3, 28]","[94, 55, 6, 17, 61, 21, 93]","[13, 101, 71, 39, 79, 24, 53]"
4,"[67, 31, 41, 100, 41, 10, 31]","[58, 31, 53, 15, 47, 15, 24]","[36, 61, 58, 58, 47, 60, 24]",None,None,None,None,None,None,None


In [40]:
course_metadata

,Course ID,Section,Teacher ID,Credit Hours,Semester
0,CE413,A,Mr. Salman Saeed,0,8
1,CE413,B,Mr. Salman Saeed,0,8
2,CS439,A,Dr. Farhan Khan,0,8
3,CS440,B,Dr. Farhan Khan,0,8
4,CS441,C,Dr. Farhan Khan,0,8
...,...,...,...,...,...
97,CS367,D,Ms. Nazia Shehzadi,0,6
98,CS373,A,Dr. Fahad Bin Muslim,0,6
99,CS374,B,Dr. Fahad Bin Muslim,0,6
100,CS375,C,Dr. Fahad Bin Muslim,0,6


In [69]:
course_metadata.to_csv('output.csv', index=False)

In [66]:
timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9
0,Row1_Col0,Row1_Col1,Row1_Col2,Row1_Col3,Row1_Col4,Row1_Col5,Row1_Col6,Row1_Col7,Row1_Col8,Row1_Col9
1,Row2_Col0,Row2_Col1,Row2_Col2,Row2_Col3,Row2_Col4,Row2_Col5,Row2_Col6,Row2_Col7,Row2_Col8,Row2_Col9
2,Row3_Col0,Row3_Col1,Row3_Col2,Row3_Col3,Row3_Col4,Row3_Col5,Row3_Col6,Row3_Col7,Row3_Col8,Row3_Col9
3,Row4_Col0,Row4_Col1,Row4_Col2,Row4_Col3,Row4_Col4,Row4_Col5,Row4_Col6,Row4_Col7,Row4_Col8,Row4_Col9
4,Row5_Col0,Row5_Col1,Row5_Col2,Row5_Col3,Row5_Col4,Row5_Col5,Row5_Col6,Row5_Col7,Row5_Col8,Row5_Col9


In [45]:
# Step 1: Count occurrences of each number in the entire DataFrame
total_counts = Counter()

# Loop through each cell and update the Counter with each list's contents
for column in timetable.columns:
    for row in timetable[column]:
        if row is not None:
            total_counts.update(row)

# Step 2: Create a DataFrame with counts of each number across all cells
# Convert the Counter to a DataFrame
count_df = pd.DataFrame(total_counts.items(), columns=['Number', 'Frequency'])

# Step 3: Sort the DataFrame by frequency in descending order
count_df = count_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)


In [46]:
count_df

,Number,Frequency
0,42,3
1,66,3
2,65,3
3,51,3
4,6,3
...,...,...
97,55,2
98,21,2
99,93,2
100,46,2


In [47]:
count_df.to_csv("jj.csv")

In [48]:


# Step 1: Count occurrences of each number in the entire DataFrame across all cells (flattened)
total_counts = Counter()

# Loop through each column and each row
for column in timetable.columns:
    for row in timetable[column]:
        if row is not None:
            total_counts.update(row)  # Update the counter with the numbers in the current list

# Step 2: Convert the counter to a DataFrame
count_df = pd.DataFrame(total_counts.items(), columns=['Number', 'Frequency'])

# Step 3: Sort the DataFrame by frequency in descending order
count_df = count_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

count_df.to_csv("cc.csv")

In [49]:
len(timetable)

5

In [71]:
temp_list = []
for i in range(0,len(timetable)):
    for lh in timetable.columns:
        temp_list.append(timetable.iloc[i][lh])

temp_list

[[29, 1, 9, 14, 64, 66, 61, 83, 86, 51],
 [77, 63, 8, 2, 95, 33, 55, 90, 6, 99],
 [79, 59, 31, 35, 49, 43, 82, 7, 85, 3],
 [62, 27, 61, 33, 81, 56, 50, 89, 0, 100],
 [4, 65, 27, 34, 101, 57, 22, 52, 81, 67],
 [14, 40, 58, 20, 69, 68, 2, 6, 11, 29],
 [94, 66, 11, 60, 97, 37, 51, 48, 53, 60],
 [77, 40, 87, 25, 100, 96, 0, 85, 67, 23],
 [34, 66, 95, 50, 50, 29, 21, 52, 38, 21],
 [67, 17, 45, 28, 62, 6, 49, 72, 77, 80],
 [62, 22, 84, 1, 1, 23, 57, 51, 30, 44],
 [36, 54, 54, 71, 33, 21, 14, 83, 59, 36],
 [98, 53, 71, 47, 35, 90, 8, 2, 54, 57],
 [5, 48, 99, 79, 28, 18, 75, 100, 69, 76],
 [16, 96, 71, 81, 7, 59, 72, 53, 68, 11],
 [22, 78, 96, 68, 25, 31, 31, 44, 87, 84],
 [91, 39, 84, 83, 34, 35, 95, 76, 91, 87],
 [69, 13, 101, 99, 82, 9, 41, 43, 12, 90],
 [70, 94, 48, 0, 43, 92, 23],
 [65, 9, 46, 79, 82, 26, 18],
 [55, 18, 25, 15, 94, 27, 91, 13, 80, 80],
 [10, 44, 13, 101, 76, 30, 37],
 [75, 15, 63, 63, 39, 40, 70],
 [32, 17, 89, 19, 64, 88, 3],
 [12, 98, 32, 12, 38, 5, 55],
 [49, 86, 78, 4

In [72]:
flattened_list = [item for sublist in temp_list if sublist is not None for item in sublist]
print(flattened_list)


[29, 1, 9, 14, 64, 66, 61, 83, 86, 51, 77, 63, 8, 2, 95, 33, 55, 90, 6, 99, 79, 59, 31, 35, 49, 43, 82, 7, 85, 3, 62, 27, 61, 33, 81, 56, 50, 89, 0, 100, 4, 65, 27, 34, 101, 57, 22, 52, 81, 67, 14, 40, 58, 20, 69, 68, 2, 6, 11, 29, 94, 66, 11, 60, 97, 37, 51, 48, 53, 60, 77, 40, 87, 25, 100, 96, 0, 85, 67, 23, 34, 66, 95, 50, 50, 29, 21, 52, 38, 21, 67, 17, 45, 28, 62, 6, 49, 72, 77, 80, 62, 22, 84, 1, 1, 23, 57, 51, 30, 44, 36, 54, 54, 71, 33, 21, 14, 83, 59, 36, 98, 53, 71, 47, 35, 90, 8, 2, 54, 57, 5, 48, 99, 79, 28, 18, 75, 100, 69, 76, 16, 96, 71, 81, 7, 59, 72, 53, 68, 11, 22, 78, 96, 68, 25, 31, 31, 44, 87, 84, 91, 39, 84, 83, 34, 35, 95, 76, 91, 87, 69, 13, 101, 99, 82, 9, 41, 43, 12, 90, 70, 94, 48, 0, 43, 92, 23, 65, 9, 46, 79, 82, 26, 18, 55, 18, 25, 15, 94, 27, 91, 13, 80, 80, 10, 44, 13, 101, 76, 30, 37, 75, 15, 63, 63, 39, 40, 70, 32, 17, 89, 19, 64, 88, 3, 12, 98, 32, 12, 38, 5, 55, 49, 86, 78, 46, 60, 74, 85, 17, 45, 74, 89, 7, 28, 24, 4, 74, 65, 30, 52, 92, 36, 64, 38,

In [73]:
occurances = Counter(flattened_list)

In [75]:
print(occurances)

Counter({29: 3, 1: 3, 9: 3, 14: 3, 64: 3, 66: 3, 61: 3, 83: 3, 86: 3, 51: 3, 77: 3, 63: 3, 8: 3, 2: 3, 95: 3, 33: 3, 55: 3, 90: 3, 6: 3, 99: 3, 79: 3, 59: 3, 31: 3, 35: 3, 49: 3, 43: 3, 82: 3, 7: 3, 85: 3, 3: 3, 62: 3, 27: 3, 81: 3, 56: 3, 50: 3, 89: 3, 0: 3, 100: 3, 4: 3, 65: 3, 34: 3, 101: 3, 57: 3, 22: 3, 52: 3, 67: 3, 40: 3, 58: 3, 20: 3, 69: 3, 68: 3, 11: 3, 94: 3, 60: 3, 97: 3, 37: 3, 48: 3, 53: 3, 87: 3, 25: 3, 96: 3, 23: 3, 21: 3, 38: 3, 17: 3, 45: 3, 28: 3, 72: 3, 80: 3, 84: 3, 30: 3, 44: 3, 36: 3, 54: 3, 71: 3, 98: 3, 47: 3, 5: 3, 18: 3, 75: 3, 76: 3, 16: 3, 91: 3, 39: 3, 13: 3, 12: 3, 70: 3, 92: 3, 26: 3, 15: 3, 10: 3, 32: 3, 19: 3, 88: 3, 74: 3, 24: 3, 73: 3, 93: 3, 78: 2, 41: 2, 46: 2, 42: 2})


In [74]:
sum = 0
for i in occurances:
    sum += occurances[i]
    
print(sum)

302
